# Формальные степенные ряды

Каждый ряд $f(x)$ -- это бесконечная последовательность коэффициентов. То есть ряд
$$
f(x) = f_0 + f_1 x + f_2 x^2 + f_3 x^3 + ...
$$
есть последовательность
$$
[f_0, f_1, f_2, f_3, ...].
$$

Последовательность мы представляем как структуру, часть вычислений которой отложенные. Так описанная выше последовательность есть `Cons(f_0, Cons(f_1, Cons(f_2, Cons(f_3, Thunk))))`. Первые четыре элемента вычислены, а для вычисления пятого надо вызвать функцию, которая этот пятый элемент посчитает.

Cons-ячейка -- это пара, которая даёт информацию о том, каково значение текущего коэффициента, и как вычислить значения всех остальных коэффициентов.

In [1]:
import Base: show, +
using Latexify

In [2]:
"Объединяем две разные структуры под одним типом"
abstract type Series{T <: Number}
end

"Cons-ячейка -- это значение текущего коэффициента и ссылка на оставшийся хвост"
struct Cons{T <: Number} <: Series{T}
    head::T
    tail::Series{T}
end

"Thunk -- это обёртка, отложенное вычисление. Функция, которая вернёт Cons-ячейку при вычислении"
struct Thunk{T <: Number} <: Series{T}
    thunk::Function
end

Thunk

In [3]:
"Деструктор. Cons-ячейку мы просто раскладвыаем в пару, а Thunk сперва вычисляем, только потом раскладываем"
function peel end

function peel(s::Cons{T})::Tuple{T,Series{T}} where T <: Number
    return (s.head, s.tail)
end

function peel(s::Thunk{T}) where T <: Number
    return peel(s.thunk())
end


"Вспомогательная функция, чтобы не писть много скобок, когда надо сделать Cons(a, Thunk(b))"
function consthunk(head::T, tail::Function)::Series{T} where T <: Number
    return Cons{T}(head, Thunk{T}(tail))
end

consthunk

In [4]:
"Конструктор степенного ряда из генерирующей функции"
function generate(::Type{T}, gen::Function)::Series{T} where T <: Number
    function run(i::Int)::Series{T}
        return consthunk(T(gen(i)), () -> run(i + 1))
    end
    
    return run(0)
end

generate

In [5]:
"Взять первые n коэффициентов и вернуть их в векторе"
function take(s::Series{T}, n::Int)::Vector{T} where T <: Number
    vec = Vector{T}(undef, n)
    
    for i in 1:n
        (vec[i], s) = peel(s)
    end
    
    return vec
end

take

In [6]:
"Преобразовать часть степенного ряда в выражение"
function series_expr(s::Series{T}, n::Int=9, var::Symbol=:x)::Expr where T <: Number
    (constant, s) = peel(s)
    args = Any[constant]
    for i in 1:n
        (coef, s) = peel(s)
        term = (i == 1) ? var : :($var ^ $i)
        term = (coef == 1) ? term : :($coef * $term)
        push!(args, term)
    end
    return Expr(:call, :+, args..., :...)
end

"Красивое представление степенного ряда в LaTeX"
function show(io::IO, ::MIME"text/latex", s::Series{T})::Nothing where T <: Number
    println(io, latexify(series_expr(s)))
end

Base.show

In [7]:
"Сложение степенных рядов"
function +(s1::Series{T}, s2::Series{T})::Series{T} where T <: Number
    (h1, t1) = peel(s1)
    (h2, t2) = peel(s2)
    return consthunk(h1 + h2, () -> t1 + t2)
end

Base.:+

In [8]:
ser1 = generate(Rational{Int}, i -> (i == 0) ? 0 : i)

Cons{Rational{Int64}}(0//1, Thunk{Rational{Int64}}(var"#1#3"{Int64}(0, Core.Box(var"#run#2"{Rational{Int64}, var"#6#7"}(var"#6#7"(), Core.Box(#= circular reference @-2 =#))))))

In [11]:
ser2 = generate(Rational{Int}, i -> (i == 0) ? 0 : (-1)^i // i)

Cons{Rational{Int64}}(0//1, Thunk{Rational{Int64}}(var"#1#3"{Int64}(0, Core.Box(var"#run#2"{Rational{Int64}, var"#12#13"}(var"#12#13"(), Core.Box(#= circular reference @-2 =#))))))

In [12]:
ser1 + ser2

Cons{Rational{Int64}}(0//1, Thunk{Rational{Int64}}(var"#4#5"{Thunk{Rational{Int64}}, Thunk{Rational{Int64}}}(Thunk{Rational{Int64}}(var"#1#3"{Int64}(0, Core.Box(var"#run#2"{Rational{Int64}, var"#12#13"}(var"#12#13"(), Core.Box(#= circular reference @-2 =#))))), Thunk{Rational{Int64}}(var"#1#3"{Int64}(0, Core.Box(var"#run#2"{Rational{Int64}, var"#6#7"}(var"#6#7"(), Core.Box(#= circular reference @-2 =#))))))))

In [ ]:
"Умножение ряда на число"
function *(a::T, s2::Series{T})::Series{T} where T <: Number
    ...
end

In [ ]:
"Умножение рядов"
function *(s1::Series{T}, s2::Series{T})::Series{T} where T <: Number
    ...
end

In [ ]:
"Обращение ряда (в смысле деления)"
function recip(s::Series{T})::Series{T} where T <: Number
    ...
end

In [ ]:
"Деление рядов"
function /(s1::Series{T}, s2::Series{T})::Series{T} where T <: Number
    ...
end

In [ ]:
"Подстановка"
function (s1::Series{T})(s2::Series{T})::Series{T} where T <: Number
    ...
end

In [ ]:
"Обращение ряда (в смысле подстановки)"
function inv(s::Series{T})::Series{T} where T <: Number
    ...
end

In [ ]:
"Дифференцирование"
function differentiate(s::Series{T})::Series{T} where T <: Number
    ...
end

In [ ]:
"Интегрирование"
function integrate(s::Series{T})::Series{T} where T <: Number
    ...
end